In [38]:
# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
import mysql.connector
from mysql.connector import errorcode


# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

In [40]:
def sacar_tuplas_moviedatabase(archivo):
    """
    Extrae las tuplas de datos de un archivo CSV.

    Parameters:
    archivo (str): La ruta del archivo CSV.

    Returns:
    list: Una lista de tuplas que contienen los datos extraídos del archivo CSV.
    """

    # Lee el archivo CSV y carga los datos en un DataFrame
    df_detalles = pd.read_csv(archivo)
    
    # Lista para almacenar las tuplas extraídas
    val = []
    
    # Lista para almacenar las filas duplicadas
    duplicadas = []

    # Recorre cada fila del DataFrame
    for pelicula in df_detalles.values:
        
        # Convierte la fila en una lista
        pelicula = list(pelicula)
        
        # Procesar los elementos de cada columna
        for i, elemento in enumerate(pelicula):
        
            # Puntuacion Rotten
            if i == 0 and not isinstance(elemento, int):
                pelicula[i] = None

            # Puntuacion IMDB
            if i == 1 and not isinstance(elemento, float) and '.' in str(elemento):
                try:
                    pelicula[i] = float(elemento)
                except ValueError:
                    pelicula[i] = None

            # Reemplaza los valores '-' con None
            if elemento == '-':
                pelicula[i] = None

            # Reemplaza los valores NaN con None
            if pd.isna(elemento):
                pelicula[i] = None

            # Iguala los tipos específicos de las películas a los más comunes
            if isinstance(elemento, str):
                if elemento == 'Short':
                    pelicula[i] = 'Cortometraje'
                elif elemento == 'Movie':
                    pelicula[i] = 'Pelicula'
                            
                elif i == 6:  # Verifica si el elemento es el género
                    # Almacenar el género como una lista para manejar múltiples géneros
                    if isinstance(elemento, str):
                        # Dividir los géneros por coma y espacio para obtener una lista de géneros
                        generos = elemento.split(', ')
                        pelicula[i] = ', '.join(generos)  # Convertir la lista en una cadena separada por comas


        # Verifica si la película ya está en la lista de datos
        if pelicula[-1] in [p[-1] for p in val]:
            duplicadas.append(pelicula)
        else:
            # Agrega la película a la lista de datos si no está duplicada
            val.append(tuple(pelicula))
    
    # Retorna la lista de tuplas
    return val


In [12]:
def sacar_tuplas_imbd(archivo):
    """
    Extrae las tuplas de datos de un archivo CSV para la tabla Detalles.

    Parameters:
        archivo (str): La ruta del archivo CSV.

    Returns:
        list: Una lista de tuplas que contienen los datos extraídos del archivo CSV.
    """

    # Lee el archivo CSV y carga los datos en un DataFrame
    df_detalles = pd.read_csv(archivo)

    # Lista para almacenar las tuplas extraídas
    val = []

    # Lista para almacenar las filas duplicadas
    duplicadas = []

    # Recorre cada fila del DataFrame
    for pelicula in df_detalles.values:

        # Convierte la fila en una lista
        pelicula = list(pelicula)

        # Reemplaza los valores '-' y NaN con None
        for i, elemento in enumerate(pelicula):
            if elemento == '-' or pd.isnull(elemento):
                pelicula[i] = None

        # Convierte los tipos de datos de las columnas
        pelicula[0] = int(pelicula[0]) if isinstance(pelicula[0], str) and pelicula[0].isdigit() else None
        pelicula[1] = float(pelicula[1]) if isinstance(pelicula[1], str) and '.' in pelicula[1] else None
        pelicula[5] = int(pelicula[5]) if isinstance(pelicula[5], str) and pelicula[5].isdigit() else None

        # Verifica si el campo IdPelicula no es nulo
        if pelicula[-1] is not None:
            # Verifica si la película ya está en la lista de datos
            if pelicula[-1] in [p[-1] for p in val]:
                duplicadas.append(pelicula)
            else:
                # Agrega la película a la lista de datos si no está duplicada
                val.append(tuple(pelicula))
        else:
            print("El campo 'IdPelicula' es nulo para la película:", pelicula)

    # Retorna la lista de tuplas
    return val


In [41]:
archivo = 'BBDD_Importaciones/Peliculas_detalles/detalles_cortos_2000_2004.csv'
prueba = sacar_tuplas_moviedatabase(archivo)


In [42]:
prueba

[(None,
  7.0,
  'Rolf Gibbs',
  None,
  'This short but lyrical piece of cinema can be described as a thriller, a horror film, a comedy, a religious epic, a tragedy, an action flick, or as a Motion Poem about 30,000 feet.',
  '5',
  'Accion',
  'g.',
  'tt0240524'),
 (None,
  None,
  'Andy Mackenzie',
  'Andy Mackenzie',
  None,
  None,
  'Accion',
  'last mistake',
  'tt0240671'),
 (None,
  6.7,
  'James Cunningham',
  'James Cunningham',
  'A mutant hero, a virtual bank heist. A digital action thriller. A three fingered virus is let loose on a computer system with the express purpose of erasing a student loan.',
  '8',
  'Accion',
  'infection',
  'tt0245967'),
 (None,
  8.5,
  "['Doug Ten', 'Napel']",
  "['Doug Ten', 'Napel']",
  "This short clay-animated film follows the two titular characters on their mission to intercept a giant robot bent on destroying their town. It doesn't go according to plan.",
  '9',
  'Accion',
  'kog-head and meatus',
  'tt0251778'),
 (None,
  6.4,
  'La

In [43]:
# Establece la conexión con la el servidor MySQL y la BBDD

cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='proyecto_cinemextract')



# Inserta los datos usando la llamada a la función en la tabla deseada

mycursor = cnx.cursor()
query = "INSERT INTO detalles (PuntuacionRottenTomatoes, PuntuacionIMDB, Direccion, Guion, Argumento, Duracion, Genero, Titulo, IdPelicula) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
#query = "INSERT INTO detalles (PuntuacionRottenTomatoes, PuntuacionIMDB, Direccion, Guion, Argumento, Duracion, Genero, Titulo, IdPelicula) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s) ON DUPLICATE KEY UPDATE PuntuacionRottenTomatoes=VALUES(PuntuacionRottenTomatoes), PuntuacionIMDB=VALUES(PuntuacionIMDB), Direccion=VALUES(Direccion), Guion=VALUES(Guion), Argumento=VALUES(Argumento), Duracion=VALUES(Duracion), Genero=VALUES(Genero), Titulo=VALUES(Titulo), IdPelicula=VALUES(IdPelicula)"

# Si el archivo de origen contiene índices, llamar a la funcion 'sacar_tuplas_i_moviedatabase'
val = sacar_tuplas_moviedatabase(archivo)

try: 
    mycursor.executemany(query, val)
    cnx.commit()
    print(mycursor.rowcount, "registro insertado.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)
    

1048 (23000): Column 'IdPelicula' cannot be null
Error Code: 1048
SQLSTATE 23000
Message Column 'IdPelicula' cannot be null
